In [ ]:
import os
com = "instalooter hashtag 여행 여행 -d -n 100000" # 10만
os.system(com)

In [24]:
import pandas as pd
def ext_split(data):
    return data.split(".")[0]
filelist = os.listdir("./여행")
filelist = list(set(pd.Series(filelist).apply(ext_split)))

In [107]:
import json, re
file_name = []
text_list = []
hashtag_list = []
for filename in tqdm(filelist):
    pattern = re.compile("#([0-9a-zA-Z가-힣]*)")
    with open(f"./여행/{filename}.json", "r") as st_json:
        
        json_file = json.load(st_json)
        try:
            split_data = json_file['edge_media_to_caption']['edges'][0]['node']['text']
        except:
            continue

        m = pattern.findall(split_data)
        
        if(len(m)==0):
            continue
        
        temp_hashtag = []
        
        
        for unit in m:
            if unit == "":
                continue
            else:
                temp_hashtag.append(unit)
        
        split_data = pattern.sub(',',split_data)
        
        file_name.append(filename)
        text_list.append(split_data)
        hashtag_list.append(temp_hashtag)

100%|██████████████████████████████████████████████| 100000/100000 [00:26<00:00, 3840.88it/s]


In [123]:
hashtag_df = pd.DataFrame(columns=['hashtag'])
for id in tqdm(range(len(hashtag_list))):
    hashtag_df.loc[id] = [hashtag_list[id]]
    
filename_df = pd.DataFrame(file_name)
text_df = pd.DataFrame(text_list)

100%|█████████████████████████████████████████████████| 65314/65314 [03:50<00:00, 282.96it/s]


In [210]:
df = pd.concat([filename_df,text_df,hashtag_df],axis=1)
df.columns=['Name','Contents','captions']

In [211]:
def delete_line(data):
    data = data.replace("\n","")
    data = data.replace("\r"," ")
    data = data.replace(",","")
    return data

find_hangul = re.compile('[가-힣]+')
def filter_hangul(data):
    if (len(find_hangul.findall(data)) == 0 or len(data)<10):
        return None
    return data


def del_ad(data):
    if "수익" in data:
        return None
    if "회원" in data:
        return None
    return data

# 불필요한 문자 제거
df.Contents = df.Contents.apply(delete_line)
        
# 광고 제거
df.Contents = df.Contents.apply(del_ad)
df.drop(df.loc[df.Contents.isnull()].index,axis=0,inplace=True)

# 한글 포함 게시글만 남기기
df.Contents = df.Contents.apply(filter_hangul)
df.drop(df.loc[df.Contents.isnull()].index,axis=0,inplace=True)

# 인덱스 초기화
df = df.reset_index(drop=True)

# 저장
df.to_csv("insta.csv",index=False,encoding='utf-8-sig')

df

,Name,Contents,captions
0,2764018393037446865,👸 468일할미가 주신 한복입고 빨빨거리고 돌아댕기다 뻐더따!집 외에는 응가 안 싸...,[많이먹어따]
1,2762426206197766802,景色最高！静かにコーヒーやお茶を楽しみたい時おすすめのカフェ♫【カフェ情報】📍 충남 아산시...,[사네]
2,2758661923951557234,우리 막내랑 예전에 카페 여행와서 둘이 뷰 보면서 뒤에서 아들들이 인생컷을 찍어줬...,"[여행, 여행스타그램, 여행그램, 야경스타그램, 감성여행, 여행중, 여행기, 여행스..."
3,2764062074039309286,.이탈리아 여행 후 그렸던 그림들.다시 한 번 잘 그려볼까 싶기도 하지만 해야할 일...,"[이탈리아, 여행, 아이패드, 푸로크리에이터, 트레이싱, Italy, 피렌체, 밀라..."
4,2761564841842721319,(한줄생략)프레딧에서 쿠폰 적용해서 산 도가니랑 설렁탕에 떡이랑 만두랑 지단 넣어서...,[온더테이블]
...,...,...,...
47378,2758628391229315782,마! 누나랑 집 가자! ❤️,"[이호테우해변, 쓰담쓰담]"
47379,2758651338748089608,미스터 선샤인 촬영지인 션샤인 랜드 에가서 메인 건물인 글로벌호텔을 담아보았는데 디...,"[논산, 선샤인스튜디오, 선샤인랜드, 여행, 여행스타그램, 여행사진, 사진계정, 사..."
47380,2758933468144356615,1/22 10시 34분 도착했는데... 예약끝났다. 휴...그래서 점심은 탕...,"[일기, 내일로, 여행, 키친마이야르, 오향가, 노티드도넛, 리베라호텔]"
47381,2763998172692098385,우리강아지~~😘😘.....,"[리케리케, 유성, 봉명동, 카페, 애견동반, 애견카페, 유성카페, 셀스타그램, 셀..."


In [215]:
# 불필요 이미지 제거
remove_file = list(set(filelist) - set(df.Name))
for target_file in tqdm(remove_file):
    os.remove(f"./여행/{target_file}.jpg")
    os.remove(f"./여행/{target_file}.json")

100%|█████████████████████████████████████████████████| 52617/52617 [02:31<00:00, 346.86it/s]
